# Critical Node Detection using CRITIC-TOPSIS Framework

This notebook provides an interactive exploration of critical node detection in complex networks.

## Contents
1. Setup and Data Loading
2. Centrality Measures Exploration
3. CRITIC Weight Analysis
4. TOPSIS Ranking
5. Attack Simulation and Evaluation
6. Visualization and Results

In [ ]:
# Setup
import sys
sys.path.insert(0, 'src')

import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Import our modules
from data_loading import (load_karate_club, load_les_miserables, 
                          load_dolphins, load_usair, get_network_info)
from centralities import compute_all_centralities
from critic import compute_critic_weights, explain_weights
from topsis import topsis_rank, get_critical_nodes
from evaluation import (compare_attack_methods, compute_attack_effectiveness,
                        get_ranking_from_centrality, get_ranking_from_topsis)
from visualization import plot_attack_curves, plot_network_with_critical_nodes

print("All modules loaded successfully!")

## 1. Load a Network

Choose one of the available networks to analyze:

In [ ]:
# Load network (change this to try different networks)
G = load_karate_club()
# G = load_les_miserables()
# G = load_dolphins()

info = get_network_info(G)
print(f"Network: {info['name']}")
print(f"Nodes: {info['nodes']}")
print(f"Edges: {info['edges']}")
print(f"Density: {info['density']:.4f}")
print(f"Average Clustering: {info['avg_clustering']:.4f}")
print(f"Average Degree: {info['avg_degree']:.2f}")

In [ ]:
# Visualize the network
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, k=2/np.sqrt(G.number_of_nodes()), seed=42)
nx.draw(G, pos, node_color='lightblue', node_size=300, 
        with_labels=True, font_size=8, edge_color='gray', alpha=0.7)
plt.title(f"{info['name']} Network")
plt.axis('off')
plt.tight_layout()
plt.show()

## 2. Compute Centrality Measures

We compute 7 different centrality measures for each node:

In [ ]:
# Compute all centralities
df_centrality = compute_all_centralities(G, verbose=True)
print(f"\nComputed {len(df_centrality.columns)} centrality measures for {len(df_centrality)} nodes")

In [ ]:
# View top nodes by each centrality
print("Top 5 nodes by each centrality:")
for col in df_centrality.columns:
    top5 = df_centrality[col].nlargest(5).index.tolist()
    print(f"  {col:15}: {top5}")

In [ ]:
# Correlation between centralities
plt.figure(figsize=(10, 8))
corr = df_centrality.corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0, square=True, linewidths=0.5)
plt.title('Correlation Between Centrality Measures')
plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- High correlation means measures capture similar information")
print("- Low correlation means measures are complementary")

## 3. CRITIC Weight Analysis

CRITIC determines objective weights based on:
- **Standard deviation** (contrast): Higher variance = more discriminating
- **Correlation** (conflict): Lower correlation = more unique information

In [ ]:
# Compute CRITIC weights
weights, critic_details = compute_critic_weights(df_centrality, verbose=True)

In [ ]:
# Visualize weights
plt.figure(figsize=(10, 6))
weights_sorted = weights.sort_values()
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(weights_sorted)))
bars = plt.barh(weights_sorted.index, weights_sorted.values, color=colors)
plt.xlabel('CRITIC Weight')
plt.title('CRITIC Weights for Each Centrality Measure')

for bar, val in zip(bars, weights_sorted.values):
    plt.text(val + 0.005, bar.get_y() + bar.get_height()/2, 
             f'{val:.3f}', va='center')

plt.xlim(0, weights_sorted.max() * 1.15)
plt.tight_layout()
plt.show()

In [ ]:
# Explain the weights
print(explain_weights(weights, critic_details))

## 4. TOPSIS Ranking

TOPSIS ranks nodes by their closeness to the ideal best and distance from the ideal worst.

In [ ]:
# Perform TOPSIS ranking
topsis_results, topsis_details = topsis_rank(df_centrality, weights, verbose=True)

In [ ]:
# View top critical nodes
print("Top 10 Critical Nodes (CRITIC-TOPSIS):")
print(topsis_results.head(10))

In [ ]:
# Visualize the network with critical nodes highlighted
critical_nodes = get_critical_nodes(topsis_results, k=10)
print(f"\nTop 10 critical nodes: {critical_nodes}")

plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, k=2/np.sqrt(G.number_of_nodes()), seed=42)

# Draw non-critical nodes
other_nodes = [n for n in G.nodes() if n not in critical_nodes]
nx.draw_networkx_nodes(G, pos, nodelist=other_nodes, node_color='lightblue',
                       node_size=200, alpha=0.6)

# Draw critical nodes
nx.draw_networkx_nodes(G, pos, nodelist=critical_nodes, node_color='red',
                       node_size=500, alpha=0.9)

nx.draw_networkx_edges(G, pos, alpha=0.3)
labels = {n: str(n) for n in critical_nodes}
nx.draw_networkx_labels(G, pos, labels, font_size=10, font_weight='bold')

plt.title(f"{info['name']} - Top 10 Critical Nodes (Red)")
plt.axis('off')
plt.tight_layout()
plt.show()

## 5. Attack Simulation

We validate our ranking by simulating targeted attacks (removing top nodes) and measuring network damage.

In [ ]:
# Create rankings for different methods
rankings = {
    'CRITIC-TOPSIS': get_ranking_from_topsis(topsis_results),
    'degree': get_ranking_from_centrality(df_centrality, 'degree'),
    'betweenness': get_ranking_from_centrality(df_centrality, 'betweenness'),
    'closeness': get_ranking_from_centrality(df_centrality, 'closeness'),
    'pagerank': get_ranking_from_centrality(df_centrality, 'pagerank'),
}

# Run attack simulation
attack_results = compare_attack_methods(G, rankings, verbose=True)

In [ ]:
# Compute and display effectiveness
effectiveness = compute_attack_effectiveness(attack_results)
print("\nAttack Effectiveness (higher = better critical node detection):")
print(effectiveness.to_string(index=False))

In [ ]:
# Plot attack curves
fig = plot_attack_curves(attack_results, title=f"{info['name']} - Targeted Attack Comparison")
plt.show()

## 6. Interpretation

### Key Questions:
1. **Which method is best?** The method with highest effectiveness (causes most damage)
2. **Does CRITIC-TOPSIS outperform single metrics?** Compare its effectiveness to others
3. **Why might results vary?** Network structure affects which centralities are informative

In [ ]:
# Summary
best_method = effectiveness.iloc[0]['method']
topsis_eff = effectiveness[effectiveness['method'] == 'CRITIC-TOPSIS']['effectiveness'].values[0]
best_eff = effectiveness.iloc[0]['effectiveness']

print(f"\n=== SUMMARY ===")
print(f"Network: {info['name']} ({info['nodes']} nodes, {info['edges']} edges)")
print(f"Best method: {best_method} (effectiveness: {best_eff:.4f})")
print(f"CRITIC-TOPSIS effectiveness: {topsis_eff:.4f}")
print(f"CRITIC-TOPSIS rank: {list(effectiveness['method']).index('CRITIC-TOPSIS') + 1} of {len(effectiveness)}")

if best_method == 'CRITIC-TOPSIS':
    print("\n✓ CRITIC-TOPSIS is the BEST method for this network!")
else:
    diff = (best_eff - topsis_eff) / best_eff * 100
    print(f"\n{best_method} outperforms CRITIC-TOPSIS by {diff:.1f}%")
    print("This may indicate single metrics capture critical structure better for this network.")

## Try Different Networks!

Go back to Section 1 and uncomment a different network loader to see how results change across different network structures.